### Constants

In [ ]:
import random
import string

PROJECT_ID = "jchavezar-demo"
REGION = "us-central1"  # @param {type: "string"}
UUID = "".join(random.choices(string.ascii_lowercase + string.digits, k=8))
BUCKET_NAME = "vtx-demos"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"
DATA_PATH = "data"
# Base
DATASET_NAME = "news_corpora"
DATASET_URI = f"{BUCKET_URI}/{DATA_PATH}/raw/newsCorpora.csv"

# Experiments
TASK = "classification"
MODEL_TYPE = "naivebayes"
EXPERIMENT_NAME = f"{TASK}-{MODEL_TYPE}-{UUID}"
EXPERIMENT_RUN_NAME = "run-1"

# Preprocessing
PREPROCESSED_DATASET_NAME = f"preprocessed_{DATASET_NAME}"
PREPROCESS_EXECUTION_NAME = "preprocess"
COLUMN_NAMES = [
    "id",
    "title",
    "url",
    "publisher",
    "category",
    "story",
    "hostname",
    "timestamp",
]
DELIMITER = "	"
INDEX_COL = 0
PREPROCESSED_DATASET_URI = (
    f"{BUCKET_URI}/{DATA_PATH}/preprocess/{PREPROCESSED_DATASET_NAME}.csv"
)

# Training
TRAIN_EXECUTION_NAME = "train"
TARGET = "category"
FEATURES = "title"
TEST_SIZE = 0.2
SEED = 8
TRAINED_MODEL_URI = f"{BUCKET_URI}/deliverables/{UUID}"
MODEL_NAME = f"{EXPERIMENT_NAME}-model"
SERVE_IMAGE = "us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-0:latest"

### Get Data 

In [ ]:
!mkdir -m 777 -p {DATA_PATH}

DATASET_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip"
!wget --no-parent {DATASET_URL} --directory-prefix={DATA_PATH}
!mkdir -m 777 -p {DATA_PATH}/temp {DATA_PATH}/raw
!unzip {DATA_PATH}/*.zip -d {DATA_PATH}/temp
!mv {DATA_PATH}/temp/*.csv {DATA_PATH}/raw && rm -Rf {DATA_PATH}/temp && rm -f {DATA_PATH}/*.zip
!gsutil -m cp -R {DATA_PATH}/raw $BUCKET_URI/{DATA_PATH}/raw

--2022-09-27 15:35:40--  https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29224203 (28M) [application/x-httpd-php]
Saving to: ‘data/NewsAggregatorDataset.zip’

NewsAggregatorDatas 100%[===================>]  27.87M  34.2MB/s    in 0.8s    

2022-09-27 15:35:41 (34.2 MB/s) - ‘data/NewsAggregatorDataset.zip’ saved [29224203/29224203]

Archive:  data/NewsAggregatorDataset.zip
  inflating: data/temp/2pageSessions.csv  
   creating: data/temp/__MACOSX/
  inflating: data/temp/__MACOSX/._2pageSessions.csv  
  inflating: data/temp/newsCorpora.csv  
  inflating: data/temp/__MACOSX/._newsCorpora.csv  
  inflating: data/temp/readme.txt    
  inflating: data/temp/__MACOSX/._readme.txt  
Copying file://data/raw/newsCorpora.csv [Content-Type=text/c

### Import Libraries

In [ ]:
# General
import logging
import collections
import tempfile
import time
from json import dumps

collections.Iterable = collections.abc.Iterable

# Vertex AI
from google.cloud import aiplatform as vertex_ai

### Initialize Vertex AI


In [ ]:
vertex_ai.init(
    project=PROJECT_ID, experiment=EXPERIMENT_NAME, staging_bucket=BUCKET_URI
)

### Initialize Experiment Run

In [ ]:
run = vertex_ai.start_run(EXPERIMENT_RUN_NAME)

Associating projects/569083142710/locations/us-central1/metadataStores/default/contexts/classification-naivebayes-kkafjjcq-run-1 to Experiment: classification-naivebayes-kkafjjcq


### Create a Dataset Metadata Artifact

In [16]:
raw_dataset_artifact = vertex_ai.Artifact.create(
    schema_title="system.Dataset", display_name=DATASET_NAME, uri=DATASET_URI
)

### Create preprocess module

In [18]:
"""
Preprocess module
"""

import pandas as pd


def preprocess(df: pd.DataFrame, text_col: str) -> pd.DataFrame:
    preprocessed_df = df.copy()
    preprocessed_df[text_col] = preprocessed_df[text_col].apply(lambda x: x.lower())
    preprocessed_df[text_col] = preprocessed_df[text_col].apply(
        lambda x: x.translate(str.maketrans("", "", string.punctuation))
    )
    return preprocessed_df

### Add preprocess execution

In [ ]:
with vertex_ai.start_execution(
    schema_title="system.ContainerExecution", display_name=PREPROCESS_EXECUTION_NAME
) as exc:
    logging.info(f"Start {PREPROCESS_EXECUTION_NAME} execution.")
    exc.assign_input_artifacts([raw_dataset_artifact])

    # Log preprocessing params --------------------------------------------------
    logging.info("Log preprocessing params.")
    vertex_ai.log_params(
        {
            "delimiter": DELIMITER,
            "features": dumps(COLUMN_NAMES),
            "index_col": INDEX_COL,
        }
    )

    # Preprocessing ------------------------------------------------------------
    logging.info("Preprocessing.")
    raw_df = pd.read_csv(
        raw_dataset_artifact.uri,
        delimiter=DELIMITER,
        names=COLUMN_NAMES,
        index_col=INDEX_COL,
    )
    preprocessed_df = preprocess(raw_df, "title")
    preprocessed_df.to_csv(PREPROCESSED_DATASET_URI, sep=",")

    # Log preprocessing metrics and store dataset artifact ---------------------
    logging.info(f"Log preprocessing metrics and {PREPROCESSED_DATASET_NAME} dataset.")
    vertex_ai.log_metrics(
        {
            "n_records": preprocessed_df.shape[0],
            "n_columns": preprocessed_df.shape[1],
        },
    )

    preprocessed_dataset_metadata = vertex_ai.Artifact.create(
        schema_title="system.Dataset",
        display_name=PREPROCESSED_DATASET_NAME,
        uri=PREPROCESSED_DATASET_URI,
    )
    exc.assign_output_artifacts([preprocessed_dataset_metadata])

### Create Model Training Model

In [1]:
"""
Train module
"""

import joblib
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


def get_training_split(
    x: pd.DataFrame, y: pd.Series, test_size: float, random_state: int
) -> (pd.DataFrame, pd.Series, pd.DataFrame, pd.Series):
    
    x_train, x_val, y_train, y_val = train_test_split(
        x, y, test_size=test_size, random_state=random_state
    )
    return x_train, x_val, y_train, y_val


def get_pipeline():
    model = Pipeline(
        [
            ("vect", CountVectorizer()),
            ("tfidf", TfidfTransformer()),
            ("clf", MultinomialNB()),
        ]
    )
    return model


def train_pipeline(model: Pipeline, X_train: pd.Series, y_train: pd.Series) -> Pipeline:
    model.fit(X_train, y_train)
    return model


def evaluate_model(model: Pipeline, X_test: pd.Series, y_test: pd.Series) -> float:
    # Evaluate model
    y_pred = model.predict(X_test)

    # Store evaluation metrics
    # Store evaluation metrics
    metrics = {
        "accuracy": round(accuracy_score(y_test, y_pred), 5),
        "precision": round(precision_score(y_test, y_pred, average="weighted"), 5),
        "recall": round(recall_score(y_test, y_pred, average="weighted"), 5),
    }
    return metrics


def save_model(model: Pipeline, save_path: str) -> int:
    try:
        with tempfile.NamedTemporaryFile() as tmp:
            joblib.dump(trained_pipeline, filename=tmp.name)
            ! gsutil cp {tmp.name} {save_path}/model.joblib
    except RuntimeError as error:
        print(error)
    return 1

### Add the train execution

In [2]:
with vertex_ai.start_execution(
    schema_title="system.ContainerExecution", display_name=TRAIN_EXECUTION_NAME
) as exc:

    exc.assign_input_artifacts([preprocessed_dataset_metadata])

    # Get training and testing data
    logging.info("Get training and testing data.")
    x_train, x_val, y_train, y_val = get_training_split(
        preprocessed_df[FEATURES],
        preprocessed_df[TARGET],
        test_size=TEST_SIZE,
        random_state=SEED,
    )
    # Get model pipeline
    logging.info("Get model pipeline.")
    pipeline = get_pipeline()

    # Log training param -------------------------------------------------------

    # Log data parameters
    logging.info("Log data parameters.")
    vertex_ai.log_params(
        {
            "target": TARGET,
            "features": FEATURES,
            "test_size": TEST_SIZE,
            "random_state": SEED,
        }
    )

    # Log pipeline parameters
    logging.info("Log pipeline parameters.")
    vertex_ai.log_params(
        {
            "pipeline_steps": dumps(
                {step[0]: str(step[1].__class__.__name__) for step in pipeline.steps}
            )
        }
    )

    # Training -----------------------------------------------------------------

    # Train model pipeline
    logging.info("Train model pipeline.")
    train_start = time.time()
    trained_pipeline = train_pipeline(pipeline, x_train, y_train)
    train_end = time.time()

    # Evaluate model
    logging.info("Evaluate model.")
    model_metrics = evaluate_model(trained_pipeline, x_val, y_val)

    # Log training metrics and store model artifact ----------------------------

    # Log training metrics
    logging.info("Log training metrics.")
    vertex_ai.log_metrics(model_metrics)

    # Generate first ten predictions
    logging.info("Generate prediction sample.")
    prediction_sample = trained_pipeline.predict(x_val)[:10]
    print("prediction sample:", prediction_sample)

    # Upload Model on Vertex AI
    logging.info("Upload Model on Vertex AI.")
    loaded = save_model(trained_pipeline, TRAINED_MODEL_URI)
    if loaded:
        model = vertex_ai.Model.upload(
            serving_container_image_uri=SERVE_IMAGE,
            artifact_uri=TRAINED_MODEL_URI,
            display_name=MODEL_NAME,
        )

    exc.assign_output_artifacts([model])

NameError: name 'vertex_ai' is not defined